In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from IPython.display import display, HTML
# init_notebook_mode(connected=False)
init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))
pd.set_option('display.max_colwidth', 100)

In [ ]:
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from evaluation_metrics import compute_metrics, compute_metrics_csv, mean_absolute_percentage_error, symetric_mean_absolute_percentage_error

In [ ]:
from statsmodels.tsa.stattools import acf, pacf, ccf, ccovf
from statsmodels.tsa.seasonal import STL
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from statsmodels.tsa.stattools import adfuller, kpss
from scipy.stats import boxcox, yeojohnson
from scipy.spatial.distance import cosine, euclidean
import rstl

In [ ]:
from scipy.stats import boxcox, yeojohnson
from dm_test import dm_test

In [ ]:
from tqdm.notebook import trange, tqdm
from datetime import datetime
import itertools
from numpy.polynomial import Polynomial as P

In [ ]:
# from lightgbm import LGBMRegressor

In [ ]:
import pickle
import calendar

In [ ]:
import ruptures as rpt
# from dtaidistance import dtw

In [ ]:
from river import tree

In [ ]:
def plot_pred_bkps(data, my_bkps, from_dt=None, title=None):
    if from_dt:
        data = data[data.index >= from_dt]
    my_bkps = [x for x in my_bkps if x < len(data)]
    
    df_err = data.copy()
    df_err.loc[:, 'AE'] = (df_err.y_true - df_err.y_pred).abs()
    df_err.loc[:, 'SE'] = (df_err.y_true - df_err.y_pred) ** 2
    df_err.loc[:, 'SAPE'] = ((df_err.y_pred - df_err.y_true).abs() / ((df_err.y_true.abs() + df_err.y_pred.abs())/2)) * 100
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data.index, y=data.y_true,
                            mode='lines',
                            line_color='#333C83', name='y_true'))
    
    fig.add_trace(go.Scatter(x=data.index, y=data.y_pred,
                            mode='lines',
                            line_color='red', name='y_pred'))
    clr_selection = 'green'
    color_switch = lambda x: 'blue' if x != 'blue' else 'green'
    
    for idx, cp in enumerate(my_bkps):
        if cp >= len(data):
            break

        fig.add_vline(x=data.index[cp], line_width=3, line_dash="dash", line_color="red")

        if idx < len(my_bkps) - 2:
            clr_selection = color_switch(clr_selection)
            fig.add_vrect(x0=data.index[cp], x1=data.index[my_bkps[idx+1]], line_width=0, fillcolor=clr_selection, opacity=0.1)
    
    fig.update_layout(title=title)
    fig.show()
    
    for roll in [24, 24*7]:
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data.index, y=df_err['SAPE'].rolling(roll).mean(),
                                mode='lines',
                                line_color='#333C83', name=f'SAPE rolling({roll})'))

        for idx, cp in enumerate(my_bkps):
                if cp >= len(data):
                    break

                fig.add_vline(x=data.index[cp], line_width=3, line_dash="dash", line_color="red")

        fig.update_layout(title=f'SAPE rolling({roll})')
        fig.show()
    
    for x in reversed(['AE', 'SE', 'SAPE']):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data.index, y=df_err[x],
                                mode='lines',
                                line_color='#333C83', name=x))

        for idx, cp in enumerate(my_bkps):
            if cp >= len(data):
                break

            fig.add_vline(x=data.index[cp], line_width=3, line_dash="dash", line_color="red")
        fig.update_layout(title=x)
        fig.show()

In [ ]:
# plot_pred_bkps(df_forecast_res, my_bkps, '2013-01-05 00:00:00')

In [ ]:
# plot_pred_bkps(df_forecast_res, my_bkps, '2014-01-01 00:00:00')

In [ ]:
# pd.set_option('display.precision', 5)
# pd.set_option('display.float_format',  '{:.3g}'.format)

In [ ]:
df = pd.read_csv('./Data/ppnet_metar_v8_2021.csv', sep=';', index_col=0)

In [ ]:
df = df[df.Year < 2021].copy()

In [ ]:
df_temp_forecast = pd.read_csv('./Data/yrno_temperature.csv', sep=';', index_col=0, decimal=',')
df_temp_forecast.index = pd.DatetimeIndex(pd.Series(df_temp_forecast.index).apply(lambda x: datetime.strptime(x, '%d.%m.%Y %H:%M')))
df_temp_forecast = df_temp_forecast.iloc[:, 1].groupby(df_temp_forecast.index).mean()

In [ ]:
df_el = pd.read_csv('./Data/raw_electricity_2021.csv', sep=',', index_col=0).Load_ffill
df_el.index = pd.DatetimeIndex(df_el.index)

In [ ]:
df.loc[:, 'TestSet'] = 0
df.loc[df.Year == 2021, 'TestSet'] = 1

In [ ]:
fig = px.line(y=df['Consumption'], x=df.index, color=df.Year, labels={'x': 'Datetime', 'y':  'Gas Consumption'})
fig.update_layout(showlegend=False, yaxis_title=r'$\text{Gas Consumption } (m^3/h)$')

fig.update_layout(
    font=dict(
        # family="Courier New, monospace",
        size=24,
        # color="RebeccaPurple"
    ),
    title_font=dict(size=24)
)
fig.show()
fig.write_image('Results/ngc_ts_plot.png', width=1920, height=600)
# fig.write_image('Results/ngc_ts_plot.pdf', width=1200, height=800)

In [ ]:
fig = px.line(y=df['Temperature'], x=df.index, color=df.Year, labels={'x': 'Datetime', 'y':  'Temperature'})
fig.update_layout(showlegend=False, yaxis_title=r'$\text{Temperature } (°C)$')

fig.update_layout(
    font=dict(
        # family="Courier New, monospace",
        size=22,
        # color="RebeccaPurple"
    ),
    title_font=dict(size=24)
)
fig.show()
fig.write_image('Results/temp_ts_plot.png', width=1920, height=600)
# fig.write_image('Results/ngc_ts_plot.pdf', width=1200, height=800)

In [ ]:
fig = px.line(y=df_el[df_el.index <= '2020-12-31 23:00:00'], x=df_el[df_el.index <= '2020-12-31 23:00:00'].index, color=df.Year, labels={'x': 'Datetime', 'y':  'Electricity load'})
fig.update_layout(showlegend=False, yaxis_title=r'$\text{Electricity load } (MW)$')

fig.update_layout(
    font=dict(
        # family="Courier New, monospace",
        size=24,
        # color="RebeccaPurple"
    ),
    title_font=dict(size=24)
)
fig.show()
fig.write_image('Results/el_ts_plot.png', width=1920, height=600)
# fig.write_image('Results/ngc_ts_plot.pdf', width=1200, height=800)

In [ ]:
gas_results_dict = {
    'SMCA': 'Results/NGC_HT_Day2Day_Online_2021.csv',
    'QDMDC': 'Results/NGC_HT_Day2Day_BySeason_2021.csv',
    'PCPDMC (Low)': 'Results/NGC_HT_Day2Day_ByCP_LOW_2021.csv',
    'PCPDMC (Medium)': 'Results/NGC_HT_Day2Day_ByCP_MED_2021.csv',
    'PCPDMC (High)': 'Results/NGC_HT_Day2Day_ByCP_HIGH_2021.csv',
    'MCPDMC (Low,WAVG)': 'Results/NGC_HT_Day2Day_ByCP_LOW_ERR_WAVG_2021.csv',
    'MCPDMC (Medium,WAVG)': 'Results/NGC_HT_Day2Day_ByCP_MED_ERR_WAVG_2021.csv',
    'MCPDMC (High,WAVG)': 'Results/NGC_HT_Day2Day_ByCP_HIGH_ERR_WAVG_2021.csv',
    'MCPDMC (Low,SWITCH)': 'Results/NGC_HT_Day2Day_ByCP_LOW_ERR_Switch_2021.csv',
    'MCPDMC (Medium,SWITCH)': 'Results/NGC_HT_Day2Day_ByCP_MED_ERR_Switch_2021.csv',
    'MCPDMC (High,SWITCH)': 'Results/NGC_HT_Day2Day_ByCP_HIGH_ERR_Switch_2021.csv',
    'FCN-L-SMCA': 'Results/NGC_FCN_BIG_Day2Day_Online.csv',
    'FCN-M-SMCA': 'Results/NGC_FCN_MEDIUM_Day2Day_Online.csv',
    'FCN-S-SMCA': 'Results/NGC_FCN_SMALL_Day2Day_Online.csv',
    'LSTM-L-SMCA': 'Results/NGC_LSTM_BIG_Day2Day_Online.csv',
    'LSTM-M-SMCA': 'Results/NGC_LSTM_MEDIUM_Day2Day_Online.csv',
    'LSTM-S-SMCA': 'Results/NGC_LSTM_SMALL_Day2Day_Online.csv',
    'GRU-L-SMCA': 'Results/NGC_GRU_BIG_Day2Day_Online.csv',
    'GRU-M-SMCA': 'Results/NGC_GRU_MEDIUM_Day2Day_Online.csv',
    'GRU-S-SMCA': 'Results/NGC_GRU_SMALL_Day2Day_Online.csv',
    'FCN-L-QDMDC': 'Results/NGC_FCN_BIG_Day2Day_BySeason_2021.csv',
    'FCN-M-QDMDC': 'Results/NGC_FCN_MEDIUM_Day2Day_BySeason_2021.csv',
    'FCN-S-QDMDC': 'Results/NGC_FCN_SMALL_Day2Day_BySeason_2021.csv',
    'LSTM-L-QDMDC': 'Results/NGC_LSTM_BIG_Day2Day_BySeason_2021.csv',
    'LSTM-M-QDMDC': 'Results/NGC_LSTM_MEDIUM_Day2Day_BySeason_2021.csv',
    'LSTM-S-QDMDC': 'Results/NGC_LSTM_SMALL_Day2Day_BySeason_2021.csv',
    'GRU-L-QDMDC': 'Results/NGC_GRU_BIG_Day2Day_BySeason_2021.csv',
    'GRU-M-QDMDC': 'Results/NGC_GRU_MEDIUM_Day2Day_BySeason_2021.csv',
    'GRU-S-QDMDC': 'Results/NGC_GRU_SMALL_Day2Day_BySeason_2021.csv',
    'FCN-L-PCPDMC (Low)': 'Results/NGC_FCN_BIG_Day2Day_ByCP_LOW_2021.csv',
    'FCN-M-PCPDMC (Low)': 'Results/NGC_FCN_MEDIUM_Day2Day_ByCP_LOW_2021.csv',
    'FCN-S-PCPDMC (Low)': 'Results/NGC_FCN_SMALL_Day2Day_ByCP_LOW_2021.csv',
    'FCN-L-PCPDMC (Medium)': 'Results/NGC_FCN_BIG_Day2Day_ByCP_MED_2021.csv',
    'FCN-M-PCPDMC (Medium)': 'Results/NGC_FCN_MEDIUM_Day2Day_ByCP_MED_2021.csv',
    'FCN-S-PCPDMC (Medium)': 'Results/NGC_FCN_SMALL_Day2Day_ByCP_MED_2021.csv',
    'FCN-L-PCPDMC (High)': 'Results/NGC_FCN_BIG_Day2Day_ByCP_HIGH_2021.csv',
    'FCN-M-PCPDMC (High)': 'Results/NGC_FCN_MEDIUM_Day2Day_ByCP_HIGH_2021.csv',
    'FCN-S-PCPDMC (High)': 'Results/NGC_FCN_SMALL_Day2Day_ByCP_HIGH_2021.csv',
    'LSTM-L-PCPDMC (Low)': 'Results/NGC_LSTM_BIG_Day2Day_ByCP_LOW_2021.csv',
    'LSTM-M-PCPDMC (Low)': 'Results/NGC_LSTM_MEDIUM_Day2Day_ByCP_LOW_2021.csv',
    'LSTM-S-PCPDMC (Low)': 'Results/NGC_LSTM_SMALL_Day2Day_ByCP_LOW_2021.csv',
    'LSTM-L-PCPDMC (Medium)': 'Results/NGC_LSTM_BIG_Day2Day_ByCP_MED_2021.csv',
    'LSTM-M-PCPDMC (Medium)': 'Results/NGC_LSTM_MEDIUM_Day2Day_ByCP_MED_2021.csv',
    'LSTM-S-PCPDMC (Medium)': 'Results/NGC_LSTM_SMALL_Day2Day_ByCP_MED_2021.csv',
    'LSTM-L-PCPDMC (High)': 'Results/NGC_LSTM_BIG_Day2Day_ByCP_HIGH_2021.csv',
    'LSTM-M-PCPDMC (High)': 'Results/NGC_LSTM_MEDIUM_Day2Day_ByCP_HIGH_2021.csv',
    'LSTM-S-PCPDMC (High)': 'Results/NGC_LSTM_SMALL_Day2Day_ByCP_HIGH_2021.csv',
    'GRU-L-PCPDMC (Low)': 'Results/NGC_GRU_BIG_Day2Day_ByCP_LOW_2021.csv',
    'GRU-M-PCPDMC (Low)': 'Results/NGC_GRU_MEDIUM_Day2Day_ByCP_LOW_2021.csv',
    'GRU-S-PCPDMC (Low)': 'Results/NGC_GRU_SMALL_Day2Day_ByCP_LOW_2021.csv',
    'GRU-L-PCPDMC (Medium)': 'Results/NGC_GRU_BIG_Day2Day_ByCP_MED_2021.csv',
    'GRU-M-PCPDMC (Medium)': 'Results/NGC_GRU_MEDIUM_Day2Day_ByCP_MED_2021.csv',
    'GRU-S-PCPDMC (Medium)': 'Results/NGC_GRU_SMALL_Day2Day_ByCP_MED_2021.csv',
    'GRU-L-PCPDMC (High)': 'Results/NGC_GRU_BIG_Day2Day_ByCP_HIGH_2021.csv',
    'GRU-M-PCPDMC (High)': 'Results/NGC_GRU_MEDIUM_Day2Day_ByCP_HIGH_2021.csv',
    'GRU-S-PCPDMC (High)': 'Results/NGC_GRU_SMALL_Day2Day_ByCP_HIGH_2021.csv',
}

In [ ]:
el_results_dict = {k: v.replace('Results/NGC', 'Results/EL').replace('ERR_Switch_2021', 'Switch_2021') for k,v in gas_results_dict.items()}


In [ ]:
results_gas = {k: pd.read_csv(v, sep=',', index_col=0) for k,v in gas_results_dict.items()}
results_el = {k: pd.read_csv(v, sep=',', index_col=0) for k,v in el_results_dict.items()}

In [ ]:
global_metrics_gas = {k: compute_metrics(v[v.index >= '2014-01-05 00:00:00'].dropna()).loc[:, ['MAE', 'MSE', 'SMAPE']] for k,v in results_gas.items()}
global_metrics_el = {k: compute_metrics(v[v.index >= '2014-01-05 00:00:00'].dropna()).loc[:, ['MAE', 'MSE', 'SMAPE']] for k,v in results_el.items()}

In [ ]:
from decimal import Decimal
pd.set_option('display.precision', 3)
pd.set_option('display.float_format',  '{:.4g}'.format)
# pd.set_option('display.float_format',  None)

In [ ]:
df_global_gas = pd.DataFrame(columns=global_metrics_gas['SMCA'].columns, index=list(global_metrics_gas.keys()))
for k,v in global_metrics_gas.items():
    df_global_gas.loc[k, :] = v.values

In [ ]:
df_global_el = pd.DataFrame(columns=global_metrics_el['SMCA'].columns, index=list(global_metrics_el.keys()))
for k,v in global_metrics_el.items():
    df_global_el.loc[k, :] = v.values

In [ ]:
df_global_gas

In [ ]:
df_global_el

In [ ]:
df_global_gas.shape, df_global_el.shape

In [ ]:
def year_metrics(df_forecast_res):
    df_pred_all = pd.DataFrame(columns=['MAE', 'MSE', 'SMAPE'], index=[x for x in range(2014, 2021)])

    for year in range(2014, 2021):
        tmp = compute_metrics(df_forecast_res[(df_forecast_res.index >= f'{year}-01-01 00:00:00') & (df_forecast_res.index <= f'{year}-12-31 00:00:00')].dropna()).loc[:, ['MAE', 'MSE', 'SMAPE']].values
        df_pred_all.loc[year, :] = tmp
    return df_pred_all

In [ ]:
yearly_metrics_gas = {k: year_metrics(v) for k,v in results_gas.items()}
yearly_metrics_el = {k: year_metrics(v) for k,v in results_el.items()}

In [ ]:
yearly_metrics_gas['SMCA']

In [ ]:
df_detail = pd.DataFrame(columns=global_metrics_gas['SMCA'].columns)
for k,v in yearly_metrics_gas.items():
    df_detail = pd.concat([df_detail, pd.DataFrame(index=[k], columns=global_metrics_gas['SMCA'].columns)])
    df_detail = pd.concat([df_detail, v])

In [ ]:
skip = 0
df_detail.iloc[8*skip:8*skip+8]

In [ ]:
df_detail

In [ ]:
yearly_metrics_gas.keys()

## Diebold-Mariano Test

In [ ]:
for k, v in results_gas.items():
    if v.iloc[:, 0].shape != v.iloc[:, 1].shape:
        print(k, v.iloc[:, 0].shape, v.iloc[:, 1].shape, v.iloc[:, 0].shape == v.iloc[:, 1].shape)

In [ ]:
for k, v in results_el.items():
    if v.iloc[:, 0].shape != v.iloc[:, 1].shape:
        print(k, v.iloc[:, 0].shape, v.iloc[:, 1].shape, v.iloc[:, 0].shape == v.iloc[:, 1].shape)

In [ ]:
df_dm_gas = pd.DataFrame(columns=results_gas.keys(), index=results_gas.keys())

for k1, v1 in tqdm(results_gas.items()):
    for k2, v2 in tqdm(results_gas.items()):
        if k1 != k2:
            try:
                # Use the same index for both series
                start_idx, end_idx = '2014-01-05 00:00:00', '2020-12-30 00:00:00'
                v1 = v1[(v1.index >= start_idx) & (v1.index <= end_idx)]
                v2 = v2[(v2.index >= start_idx) & (v2.index <= end_idx)]
                idx = v1.index.intersection(v2.index)
                v1 = v1.loc[idx]
                v2 = v2.loc[idx]
                df_dm_gas.loc[k1, k2] = dm_test(v1.y_true, v1.y_pred, v2.y_pred, h = 24)
                df_dm_gas.loc[k2, k1] = dm_test(v1.y_true, v1.y_pred, v2.y_pred, h = 24)
            except Exception as e:
                print(k1, k2, v1.dtypes, v2.dtypes, v1.shape, v2.shape, e)
            

In [ ]:
df_dm_gas.to_csv('Results/dm_gas.csv', sep=';')

In [ ]:
def map_tuples_to_p_values(x):
    if type(x) == float:
        return x
    else:
        x = x[1]
        return x

In [ ]:
df_dm_gas_post_process = df_dm_gas.copy()
df_dm_gas_post_process = df_dm_gas_post_process.applymap(map_tuples_to_p_values)
selected_models = ['SMCA', 'QDMDC', 'FCN-M-SMCA', 'FCN-M-QDMDC', 'FCN-M-PCPDMC (Low)', 'MCPDMC (Low,WAVG)', 'PCPDMC (Low)', 'PCPDMC (Medium)', 'PCPDMC (High)']
df_dm_gas_post_process = df_dm_gas_post_process.loc[selected_models, selected_models]

rename_dict = {
    'SMCA': 'HT-SMCA',
    'QDMDC': 'HT-QDMDC',
    'PCPDMC (Low)': 'HT-PCPDMC (Low)',
    'MCPDMC (Low,WAVG)': 'HT-MCPDMC-WA (Low)',
    'MCPDMC (Low,SWITCH)': 'HT-MCPDMC-SW',
    'PCPDMC (Medium)': 'HT-PCPDMC (Medium)',
    'MCPDMC (Medium,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (Medium,SWITCH)': 'HT-MCPDMC-SW',
    'PCPDMC (High)': 'HT-PCPDMC (High)',
    'MCPDMC (High,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (High,SWITCH)': 'HT-MCPDMC-SW',
    'FCN-M-SMCA': 'FCN-SMCA (M)',
    'FCN-M-QDMDC': 'FCN-QDMDC (M)',
    'FCN-M-PCPDMC (Low)': 'FCN-PCPDMC (M, Low)',
    'FCN-M-PCPDMC (Medium)': 'FCN-PCPDMC (M, Medium)',
    'FCN-M-PCPDMC (High)': 'FCN-PCPDMC (M, High)',
}



fig = plt.figure(figsize=(8, 8))
sns.heatmap(df_dm_gas_post_process.rename(index=rename_dict, columns=rename_dict, inplace=False).astype(float), annot=True, cmap='Greens', fmt=".2g", cbar=False, linewidths=0.1, linecolor='black', annot_kws={"size": 10}, square=True, vmin=0, vmax=0.05)
plt.show()
fig.tight_layout()
fig.savefig('Results/dm_gas_heatmap.png', bbox_inches='tight')

In [ ]:
df_dm_el = pd.read_csv('Results/dm_el_post_process.csv', sep=';', index_col=0)
df_dm_el_post_process = df_dm_el.copy()

selected_models = ['SMCA', 'QDMDC', 'FCN-S-SMCA', 'FCN-M-QDMDC', 'FCN-M-PCPDMC (Low)', 'PCPDMC (Low)', 'PCPDMC (Medium)', 'PCPDMC (High)']
df_dm_el_post_process = df_dm_el_post_process.loc[selected_models, selected_models]

rename_dict = {
    'SMCA': 'HT-SMCA',
    'QDMDC': 'HT-QDMDC',
    'PCPDMC (Low)': 'HT-PCPDMC (Low)',
    'MCPDMC (Low,WAVG)': 'HT-MCPDMC-WA (Low)',
    'MCPDMC (Low,SWITCH)': 'HT-MCPDMC-SW',
    'PCPDMC (Medium)': 'HT-PCPDMC (Medium)',
    'MCPDMC (Medium,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (Medium,SWITCH)': 'HT-MCPDMC-SW',
    'PCPDMC (High)': 'HT-PCPDMC (High)',
    'MCPDMC (High,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (High,SWITCH)': 'HT-MCPDMC-SW',
    'FCN-M-SMCA': 'FCN-SMCA (M)',
    'FCN-S-SMCA': 'FCN-SMCA (S)',
    'FCN-M-QDMDC': 'FCN-QDMDC (M)',
    'FCN-M-PCPDMC (Low)': 'FCN-PCPDMC (M, Low)',
    'FCN-M-PCPDMC (Medium)': 'FCN-PCPDMC (M, Medium)',
    'FCN-M-PCPDMC (High)': 'FCN-PCPDMC (M, High)',
}



fig = plt.figure(figsize=(8, 8))
sns.heatmap(df_dm_el_post_process.rename(index=rename_dict, columns=rename_dict, inplace=False).astype(float), annot=True, cmap='Greens', fmt=".2g", cbar=False, linewidths=0.1, linecolor='black', annot_kws={"size": 10}, square=True, vmin=0, vmax=0.05)
plt.show()
fig.tight_layout()
fig.savefig('Results/dm_el_heatmap.png', bbox_inches='tight')

In [ ]:
df_dm_gas_post_process = df_dm_gas.copy()
df_dm_gas_post_process = df_dm_gas_post_process.applymap(map_tuples_to_p_values)
selected_models = ['SMCA', 'QDMDC', 'FCN-M-SMCA', 'FCN-M-QDMDC', 'FCN-M-PCPDMC (Low)', 'MCPDMC (Low,WAVG)', 'PCPDMC (Low)', 'PCPDMC (Medium)', 'PCPDMC (High)']
df_dm_gas_post_process = df_dm_gas_post_process.loc[selected_models, selected_models]

rename_dict = {
    'SMCA': 'HT-SMCA',
    'QDMDC': 'HT-QDMDC',
    'PCPDMC (Low)': 'HT-PCPDMC (Low)',
    'MCPDMC (Low,WAVG)': 'HT-MCPDMC-WA (Low)',
    'MCPDMC (Low,SWITCH)': 'HT-MCPDMC-SW',
    'PCPDMC (Medium)': 'HT-PCPDMC (Medium)',
    'MCPDMC (Medium,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (Medium,SWITCH)': 'HT-MCPDMC-SW',
    'PCPDMC (High)': 'HT-PCPDMC (High)',
    'MCPDMC (High,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (High,SWITCH)': 'HT-MCPDMC-SW',
    'FCN-M-SMCA': 'FCN-SMCA (M)',
    'FCN-M-QDMDC': 'FCN-QDMDC (M)',
    'FCN-M-PCPDMC (Low)': 'FCN-PCPDMC (M, Low)',
    'FCN-M-PCPDMC (Medium)': 'FCN-PCPDMC (M, Medium)',
    'FCN-M-PCPDMC (High)': 'FCN-PCPDMC (M, High)',
}



fig = plt.figure(figsize=(8, 8))
sns.heatmap(df_dm_gas_post_process.rename(index=rename_dict, columns=rename_dict, inplace=False).astype(float), annot=True, cmap='Greens', fmt=".2g", cbar=False, linewidths=0.1, linecolor='black', annot_kws={"size": 10}, square=True, vmin=0, vmax=0.05)
plt.show()
fig.tight_layout()
fig.savefig('Results/dm_gas_heatmap.png', bbox_inches='tight')

In [ ]:
fig = go.Figure()

mode = 'High'
metric = 'SMAPE'
keys_dict = {
    'Low': ['SMCA', 'QDMDC', 'PCPDMC (Low)', 'MCPDMC (Low,WAVG)', 'MCPDMC (Low,SWITCH)', 'FCN-M-SMCA', 'FCN-M-QDMDC', 'FCN-M-PCPDMC (Low)'],
    'Medium': ['SMCA', 'QDMDC', 'PCPDMC (Medium)', 'MCPDMC (Medium,WAVG)', 'MCPDMC (Medium,SWITCH)', 'FCN-M-SMCA', 'FCN-M-QDMDC', 'FCN-M-PCPDMC (Medium)'],
    'High': ['SMCA', 'QDMDC', 'PCPDMC (High)', 'MCPDMC (High,WAVG)', 'MCPDMC (High,SWITCH)', 'FCN-M-SMCA', 'FCN-M-QDMDC', 'FCN-M-PCPDMC (High)'],
}
# keys = [x for x in yearly_metrics_gas.keys() if mode in x or 'SMCA' in x or 'QDMDC' in x]
keys = keys_dict[mode]
years = yearly_metrics_gas['SMCA'].index

rename_dict = {
    'SMCA': 'HT-SMCA',
    'QDMDC': 'HT-QDMDC',
    'PCPDMC (Low)': 'HT-PCPDMC',
    'MCPDMC (Low,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (Low,SWITCH)': 'HT-MCPDMC-SW',
    'PCPDMC (Medium)': 'HT-PCPDMC',
    'MCPDMC (Medium,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (Medium,SWITCH)': 'HT-MCPDMC-SW',
    'PCPDMC (High)': 'HT-PCPDMC',
    'MCPDMC (High,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (High,SWITCH)': 'HT-MCPDMC-SW',
    'FCN-M-SMCA': 'FCN-SMCA (M)',
    'FCN-M-QDMDC': 'FCN-QDMDC (M)',
    'FCN-M-PCPDMC (Low)': 'FCN-PCPDMC (M)',
    'FCN-M-PCPDMC (Medium)': 'FCN-PCPDMC (M)',
    'FCN-M-PCPDMC (High)': 'FCN-PCPDMC (M)',
}

for k in keys:
    data = yearly_metrics_gas[k]
    # name = k.replace(f'({mode},', '').replace(f'({mode})', '').replace(f' WAVG)', '-WA').replace(f' SWITCH)', '-SW')
    name = rename_dict[k]
    fig.add_trace(go.Scatter(x=years, y=data[metric],
                        mode='lines+markers',
                        name=name))
fig.update_layout(showlegend=True, xaxis_title='Year', yaxis_title=f'{metric} (%)', margin=dict(l=0, r=0, t=0, b=0), legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    )
)

fig.show()
fig.write_image(f'Results/ngc_smape_progress_{mode}.png', width=1450, height=500)

In [ ]:
fig = go.Figure()

mode = 'High'
metric = 'SMAPE'
keys_dict = {
    'Low': ['SMCA', 'QDMDC', 'PCPDMC (Low)', 'MCPDMC (Low,WAVG)', 'MCPDMC (Low,SWITCH)', 'FCN-S-SMCA', 'FCN-M-QDMDC', 'FCN-M-PCPDMC (Low)'],
    'Medium': ['SMCA', 'QDMDC', 'PCPDMC (Medium)', 'MCPDMC (Medium,WAVG)', 'MCPDMC (Medium,SWITCH)', 'FCN-S-SMCA', 'FCN-M-QDMDC', 'FCN-L-PCPDMC (Medium)'],
    'High': ['SMCA', 'QDMDC', 'PCPDMC (High)', 'MCPDMC (High,WAVG)', 'MCPDMC (High,SWITCH)', 'FCN-S-SMCA', 'FCN-M-QDMDC', 'FCN-M-PCPDMC (High)'],
}
# keys = [x for x in yearly_metrics_gas.keys() if mode in x or 'SMCA' in x or 'QDMDC' in x]
keys = keys_dict[mode]
years = yearly_metrics_gas['SMCA'].index

rename_dict = {
    'SMCA': 'HT-SMCA',
    'QDMDC': 'HT-QDMDC',
    'PCPDMC (Low)': 'HT-PCPDMC',
    'MCPDMC (Low,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (Low,SWITCH)': 'HT-MCPDMC-SW',
    'PCPDMC (Medium)': 'HT-PCPDMC',
    'MCPDMC (Medium,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (Medium,SWITCH)': 'HT-MCPDMC-SW',
    'PCPDMC (High)': 'HT-PCPDMC',
    'MCPDMC (High,WAVG)': 'HT-MCPDMC-WA',
    'MCPDMC (High,SWITCH)': 'HT-MCPDMC-SW',
    'FCN-S-SMCA': 'FCN-SMCA (S)',
    'FCN-M-QDMDC': 'FCN-QDMDC (M)',
    'FCN-M-PCPDMC (Low)': 'FCN-PCPDMC (M)',
    'FCN-L-PCPDMC (Medium)': 'FCN-PCPDMC (L)',
    'FCN-M-PCPDMC (High)': 'FCN-PCPDMC (M)',
}

for k in keys:
    data = yearly_metrics_el[k]
    # name = k.replace(f'({mode},', '').replace(f'({mode})', '').replace(f' WAVG)', '-WA').replace(f' SWITCH)', '-SW')
    name = rename_dict[k]
    fig.add_trace(go.Scatter(x=years, y=data[metric],
                        mode='lines+markers',
                        name=name))
fig.update_layout(showlegend=True, xaxis_title='Year', yaxis_title=f'{metric} (%)', margin=dict(l=0, r=0, t=0, b=0), legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.update_layout(
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    )
)

fig.show()
fig.write_image(f'Results/el_smape_progress_{mode}.png', width=1450, height=500)

In [ ]:
y_true = results_gas['PCPDMC (Low)'].loc[results_gas['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_true
y_pred_PCPDMC = results_gas['PCPDMC (Low)'].loc[results_gas['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_pred
y_pred_MCPDMC_WA = results_gas['MCPDMC (High,WAVG)'].loc[results_gas['MCPDMC (High,WAVG)'].index >= '2020-01-01 00:00:00'].y_pred
y_pred_MCPDMC_SW = results_gas['MCPDMC (High,SWITCH)'].loc[results_gas['MCPDMC (High,SWITCH)'].index >= '2020-01-01 00:00:00'].y_pred
y_pred_MCPDMC_SW = results_gas['MCPDMC (High,SWITCH)'].loc[results_gas['MCPDMC (High,SWITCH)'].index >= '2020-01-01 00:00:00'].y_pred
y_pred_FCN_M_SMCA = results_gas['FCN-M-SMCA'].loc[results_gas['FCN-M-SMCA'].index >= '2020-01-01 00:00:00'].y_pred
y_pred_LSTM_L_QDMDC = results_gas['LSTM-L-QDMDC'].loc[results_gas['LSTM-L-QDMDC'].index >= '2020-01-01 00:00:00'].y_pred

# LSTM-S-PCPDMC (Medium), LSTM-QDMDC (L)


y_pred_FCN_M_SMCA

In [ ]:
bkps_low = pd.read_csv('Results/BKPS_LOW.csv')
bkps_low = bkps_low.iloc[:4]
bkps_low.DateTime = bkps_low.DateTime.apply(lambda x: x.replace('2013', '2020'))
bkps_low

In [ ]:
bkps_high = pd.read_csv('Results/BKPS_HIGH.csv')
bkps_high = bkps_high.iloc[:13]
bkps_high.DateTime = bkps_high.DateTime.apply(lambda x: x.replace('2013', '2020'))
bkps_high

In [ ]:
fig = go.Figure()

plot_data_kw = {'PCPDMC (Low)': y_pred_PCPDMC, 'MCPDMC-WA (High)': y_pred_MCPDMC_WA,'MCPDMC-SW (High)':y_pred_MCPDMC_SW , 'FCN-SMCA (M)': y_pred_FCN_M_SMCA, 'Ground Truth':  y_true}
keys = plot_data_kw.keys()
for k, v in plot_data_kw.items():
    data = v
    fig.add_trace(go.Scatter(x=data.index, y=data.values,
                        mode='lines',
                        name=k))

for x in bkps_low.DateTime:
    fig.add_vline(x=x, line_width=3, line_dash="dash", line_color="red")
    
for x in bkps_high.DateTime:
    fig.add_vline(x=x, line_width=3, line_dash="dot", line_color="black", opacity=0.5)
    
# r'$\text{Gas Consumption } (m^3/h)$'
fig.update_layout(showlegend=True, xaxis_title='Datetime', yaxis_title='Gas Consumption', margin=dict(l=0, r=0, t=0, b=0),
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
))
fig.show()
fig.write_image(f'Results/ngc_prediction_2020_v2.png', width=1450, height=500)

In [ ]:
fig = go.Figure()


plot_data_kw = {'HT-PCPDMC (Low)': y_pred_PCPDMC, 'FCN-SMCA (M)': y_pred_FCN_M_SMCA,'LSTM-QDMDC (L)': y_pred_LSTM_L_QDMDC , 'Ground Truth':  y_true}
keys = plot_data_kw.keys()
for k, v in plot_data_kw.items():
    from_dt, to_dt = '2020-03-29 00:00:00', '2020-04-18 00:00:00'
    data = v.loc[(v.index >= from_dt) & (v.index <= to_dt)]
    fig.add_trace(go.Scatter(x=data.index, y=data.values,
                        mode='lines',
                        name=k))

for x in bkps_low.DateTime:
    if x >= from_dt and x <= to_dt:
        fig.add_vline(x=x, line_width=3, line_dash="dash", line_color="red")
    
for x in bkps_high.DateTime:
    if x >= from_dt and x <= to_dt:
        fig.add_vline(x=x, line_width=3, line_dash="dot", line_color="black", opacity=0.5)
    
# r'$\text{Gas Consumption } (m^3/h)$'
fig.update_layout(showlegend=True, xaxis_title='Datetime', yaxis_title='Gas Consumption', margin=dict(l=0, r=0, t=0, b=0),
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ), 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()
fig.write_image(f'Results/ngc_pred_fail_2020_v2.png', width=1450, height=500)

In [ ]:
fig = go.Figure()

plot_data_kw = {'HT-PCPDMC (Low)': y_pred_PCPDMC, 'HT-MCPDMC-WA (High)': y_pred_MCPDMC_WA,'MCPDMC-SW (High)':y_pred_MCPDMC_SW , 'FCN-SMCA (M)': y_pred_FCN_M_SMCA, 'Ground Truth':  y_true}
keys = plot_data_kw.keys()
for k, v in plot_data_kw.items():
    data = v.loc[(v.index >= '2020-03-26 00:00:00') & (v.index <= '2020-04-20 00:00:00')]
    fig.add_trace(go.Scatter(x=data.index, y=data.values,
                        mode='lines',
                        name=k))

for x in bkps_low.DateTime:
    if x >= '2020-03-26 00:00:00' and x <= '2020-04-20 00:00:00':
        fig.add_vline(x=x, line_width=3, line_dash="dash", line_color="red")
    
for x in bkps_high.DateTime:
    if x >= '2020-03-26 00:00:00' and x <= '2020-04-20 00:00:00':
        fig.add_vline(x=x, line_width=3, line_dash="dot", line_color="black", opacity=0.5)
    
# r'$\text{Gas Consumption } (m^3/h)$'
fig.update_layout(showlegend=True, xaxis_title='Datetime', yaxis_title='Gas Consumption', margin=dict(l=0, r=0, t=0, b=0),
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ), 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()
fig.write_image(f'Results/ngc_pred_example_2020_v2.png', width=1450, height=500)

In [ ]:
y_true = results_gas['PCPDMC (Low)'].loc[results_gas['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_true

y_pred = results_gas['PCPDMC (Low)'].loc[results_gas['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_pred
smape_PCPDMC = np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0)) * 100

y_pred = results_gas['MCPDMC (High,WAVG)'].loc[results_gas['MCPDMC (High,WAVG)'].index >= '2020-01-01 00:00:00'].y_pred
smape_MCPDMC_WA = np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0)) * 100

y_pred = results_gas['MCPDMC (High,SWITCH)'].loc[results_gas['MCPDMC (High,SWITCH)'].index >= '2020-01-01 00:00:00'].y_pred
smape_MCPDMC_SW = np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0)) * 100

y_pred = results_gas['FCN-M-SMCA'].loc[results_gas['FCN-M-SMCA'].index >= '2020-01-01 00:00:00'].y_pred
smape_FCN_M_SMCA = np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0)) * 100

smape_FCN_M_SMCA

In [ ]:
df_sape = pd.DataFrame(index=y_true.index)
df_sape['PCPDMC (Low)'] = smape_PCPDMC
df_sape['MCPDMC-WA (High)'] = smape_MCPDMC_WA
df_sape['MCPDMC-SW (High)'] = smape_MCPDMC_SW
df_sape['FCN-SMCA (M)'] = smape_FCN_M_SMCA
df_sape['Month'] = pd.DatetimeIndex(df_sape.index).month
df_sape

In [ ]:
fig = go.Figure()

plot_data_kw = {'HT-PCPDMC (Low)': smape_PCPDMC, 'HT-MCPDMC-WA (High)': smape_MCPDMC_WA, 'HT-MCPDMC-SW (High)':smape_MCPDMC_SW, 'FCN-SMCA (M)': smape_FCN_M_SMCA}
keys = plot_data_kw.keys()
for k, v in plot_data_kw.items():
    data = v
    fig.add_trace(go.Scatter(x=data.index, y=data.values,
                        mode='lines',
                        name=k))

for x in bkps_low.DateTime:
    fig.add_vline(x=x, line_width=3, line_dash="dash", line_color="red")
    
for x in bkps_high.DateTime:
    fig.add_vline(x=x, line_width=3, line_dash="dot", line_color="black", opacity=0.5)
    
fig.update_layout(showlegend=True, xaxis_title='Datetime', yaxis_title='SAPE (%)', margin=dict(l=0, r=0, t=0, b=0), 
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ), 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()
fig.write_image(f'Results/ngc_sape_2020_v2.png', width=1450, height=500)

In [ ]:
df_sape['Month']

In [ ]:
df_sape_s = df_sape.iloc[:, :4].stack().reset_index(name='SAPE').rename({'level_1': 'Approach'}, axis=1)
df_sape_s['Month'] = pd.DatetimeIndex(df_sape_s['level_0']).month
df_sape_s.Approach = df_sape_s.Approach.replace({'PCPDMC (Low)': 'HT-PCPDMC (Low)', 'MCPDMC-WA (High)': 'HT-MCPDMC-WA (High)', 'MCPDMC-SW (High)': 'HT-MCPDMC-SW (High)	'})
df_sape_s

In [ ]:
np.arange(1, 13)

In [ ]:
[calendar.month_name[x] for x in np.arange(1, 13)]

In [ ]:
fig = px.box(df_sape_s, x='Month', y='SAPE', color='Approach')
fig.update_layout(showlegend=True, xaxis_title='Month', yaxis_title='SAPE (%)', margin=dict(l=0, r=0, t=0, b=0), legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
),
font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ),                 
xaxis = dict(
tickmode = 'array',
tickvals = np.arange(1, 13),
ticktext = [calendar.month_name[x] for x in np.arange(1, 13)]
))
fig.show()
fig.write_image(f'Results/ngc_sape_boxplot_2020_v2.png', width=1450, height=500)

In [ ]:
df_sape_agg = df_sape.groupby('Month').median()
df_sape_agg.index = [calendar.month_name[x] for x in df_sape_agg.index]
df_sape_agg

## Electricity

In [ ]:
y_true = results_el['PCPDMC (Low)'].loc[results_el['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_true
y_pred_PCPDMC = results_el['PCPDMC (Low)'].loc[results_el['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_pred
y_pred_MCPDMC_WA = results_el['MCPDMC (High,WAVG)'].loc[results_el['MCPDMC (High,WAVG)'].index >= '2020-01-01 00:00:00'].y_pred
y_pred_MCPDMC_SW = results_el['MCPDMC (High,SWITCH)'].loc[results_el['MCPDMC (High,SWITCH)'].index >= '2020-01-01 00:00:00'].y_pred
y_pred_MCPDMC_SW = results_el['MCPDMC (High,SWITCH)'].loc[results_el['MCPDMC (High,SWITCH)'].index >= '2020-01-01 00:00:00'].y_pred
y_pred_FCN_S_SMCA = results_el['FCN-S-SMCA'].loc[results_el['FCN-S-SMCA'].index >= '2020-01-01 00:00:00'].y_pred

y_pred_FCN_S_SMCA

In [ ]:
bkps_low = pd.read_csv('Results/EL_BKPS_LOW.csv')
bkps_low = bkps_low.iloc[1:6]
bkps_low.Datetime = bkps_low.Datetime.apply(lambda x: x.replace('2013', '2020'))
bkps_low

In [ ]:
bkps_high = pd.read_csv('Results/EL_BKPS_HIGH.csv')
bkps_high = bkps_high.iloc[1:13]
bkps_high.Datetime = bkps_high.Datetime.apply(lambda x: x.replace('2013', '2020'))
bkps_high

In [ ]:
fig = go.Figure()

plot_data_kw = {'HT-PCPDMC (Low)': y_pred_PCPDMC, 'HT-MCPDMC-WA (High)': y_pred_MCPDMC_WA,'HT-MCPDMC-SW (High)':y_pred_MCPDMC_SW, 'FCN-SMCA (S)': y_pred_FCN_S_SMCA, 'Ground Truth':  y_true}
keys = plot_data_kw.keys()
for k, v in plot_data_kw.items():
    data = v
    fig.add_trace(go.Scatter(x=data.index, y=data.values,
                        mode='lines',
                        name=k))

for x in bkps_low.Datetime:
    fig.add_vline(x=x, line_width=3, line_dash="dash", line_color="red")
    
for x in bkps_high.Datetime:
    fig.add_vline(x=x, line_width=3, line_dash="dot", line_color="black", opacity=0.5)
    
# r'$\text{Gas Consumption } (m^3/h)$'
fig.update_layout(showlegend=True, xaxis_title='Datetime', yaxis_title='Electricity load', margin=dict(l=0, r=0, t=0, b=0),
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ),
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
))
fig.show()
fig.write_image(f'Results/el_prediction_2020_v2.png', width=1450, height=500)

In [ ]:
fig = go.Figure()

plot_data_kw = {'HT-PCPDMC (Low)': y_pred_PCPDMC, 'HT-MCPDMC-WA (High)': y_pred_MCPDMC_WA,'HT-MCPDMC-SW (High)':y_pred_MCPDMC_SW , 'FCN-SMCA (S)': y_pred_FCN_S_SMCA, 'Ground Truth':  y_true}
keys = plot_data_kw.keys()
for k, v in plot_data_kw.items():
    # dt_from, dt_to = '2020-03-26 00:00:00', '2020-04-20 00:00:00'
    dt_from, dt_to = '2020-01-08 00:00:00', '2020-02-04 00:00:00'
    data = v.loc[(v.index >= dt_from) & (v.index <= dt_to)]
    fig.add_trace(go.Scatter(x=data.index, y=data.values,
                        mode='lines',
                        name=k))

for x in bkps_low.Datetime:
    if x >= dt_from and x <= dt_to:
        fig.add_vline(x=x, line_width=3, line_dash="dash", line_color="red")
    
for x in bkps_high.Datetime:
    if x >= dt_from and x <= dt_to:
        fig.add_vline(x=x, line_width=3, line_dash="dot", line_color="black", opacity=0.5)
    
# r'$\text{Gas Consumption } (m^3/h)$'
fig.update_layout(showlegend=True, xaxis_title='Datetime', yaxis_title='Gas Consumption', margin=dict(l=0, r=0, t=0, b=0),
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ), 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()
fig.write_image(f'Results/el_pred_example_2020_v2.png', width=1450, height=500)

In [ ]:
y_true = results_el['PCPDMC (Low)'].loc[results_el['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_true

y_pred = results_el['PCPDMC (Low)'].loc[results_el['PCPDMC (Low)'].index >= '2020-01-01 00:00:00'].y_pred
smape_PCPDMC = np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0)) * 100

y_pred = results_el['MCPDMC (High,WAVG)'].loc[results_el['MCPDMC (High,WAVG)'].index >= '2020-01-01 00:00:00'].y_pred
smape_MCPDMC_WA = np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0)) * 100

y_pred = results_el['MCPDMC (High,SWITCH)'].loc[results_el['MCPDMC (High,SWITCH)'].index >= '2020-01-01 00:00:00'].y_pred
smape_MCPDMC_SW = np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0)) * 100

y_pred = results_el['FCN-S-SMCA'].loc[results_el['FCN-S-SMCA'].index >= '2020-01-01 00:00:00'].y_pred
smape_FCN_S_SMCA = np.abs((y_pred - y_true) / ((np.abs(y_true) + np.abs(y_pred))/2.0)) * 100

smape_FCN_S_SMCA

In [ ]:
df_sape = pd.DataFrame(index=y_true.index)
df_sape['PCPDMC (Low)'] = smape_PCPDMC
df_sape['MCPDMC-WA (High)'] = smape_MCPDMC_WA
df_sape['MCPDMC-SW (High)'] = smape_MCPDMC_SW
df_sape['FCN-SMCA (S)'] = smape_FCN_S_SMCA
df_sape['Month'] = pd.DatetimeIndex(df_sape.index).month
df_sape

In [ ]:
fig = go.Figure()

plot_data_kw = {'HT-PCPDMC (Low)': smape_PCPDMC, 'HT-MCPDMC-WA (High)': smape_MCPDMC_WA, 'HT-MCPDMC-SW (High)':smape_MCPDMC_SW, 'FCN-SMCA (S)': smape_FCN_S_SMCA}
keys = plot_data_kw.keys()
for k, v in plot_data_kw.items():
    data = v
    fig.add_trace(go.Scatter(x=data.index, y=data.values,
                        mode='lines',
                        name=k))

for x in bkps_low.Datetime:
    fig.add_vline(x=x, line_width=3, line_dash="dash", line_color="red")
    
for x in bkps_high.Datetime:
    fig.add_vline(x=x, line_width=3, line_dash="dot", line_color="black", opacity=0.5)
    
fig.update_layout(showlegend=True, xaxis_title='Datetime', yaxis_title='SAPE (%)', margin=dict(l=0, r=0, t=0, b=0), 
    font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ), 
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()
fig.write_image(f'Results/el_sape_2020_v2.png', width=1450, height=500)

In [ ]:
df_sape['Month']

In [ ]:
df_sape_s = df_sape.iloc[:, :4].stack().reset_index(name='SAPE').rename({'level_1': 'Approach'}, axis=1)
df_sape_s['Month'] = pd.DatetimeIndex(df_sape_s['level_0']).month
df_sape_s.Approach = df_sape_s.Approach.replace({'PCPDMC (Low)': 'HT-PCPDMC (Low)', 'MCPDMC-WA (High)': 'HT-MCPDMC-WA (High)', 'MCPDMC-SW (High)': 'HT-MCPDMC-SW (High)	'})
df_sape_s

In [ ]:
np.arange(1, 13)

In [ ]:
[calendar.month_name[x] for x in np.arange(1, 13)]

In [ ]:
fig = px.box(df_sape_s, x='Month', y='SAPE', color='Approach')
fig.update_layout(showlegend=True, xaxis_title='Month', yaxis_title='SAPE (%)', margin=dict(l=0, r=0, t=0, b=0), legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1,
),
font=dict(
        # family="Courier New, monospace",
        size=20,
        # color="RebeccaPurple"
    ),                 
xaxis = dict(
tickmode = 'array',
tickvals = np.arange(1, 13),
ticktext = [calendar.month_name[x] for x in np.arange(1, 13)]
))
fig.show()
fig.write_image(f'Results/el_sape_boxplot_2020_v2.png', width=1450, height=500)

In [ ]:
df_sape_agg = df_sape.groupby('Month').median()
df_sape_agg.index = [calendar.month_name[x] for x in df_sape_agg.index]
df_sape_agg.apply(lambda x: round(x, 2))